# 1.

* 下記の条件での、ホップごとの遅延 (per-hop delay) の値はいくつになりますか？

  * 処理遅延 (Processing delay) = $0.1$ [ms]
  * キューイング遅延 (Queueing delay) = $20$ [ms]
  * リンクの距離 (Link distance) = $200$ [km]
  * リンクの伝搬スピード (Link propagation speed) = $(2 \times 10^8)$ [m/s]
  * パケットサイズ (Packet size) = $1500$ [bytes]
  * リンク伝送速度 (Link transmission rate) = $100$ [Mbps]

* 途中の計算式も含めて，計算結果を回答してください /

In [8]:
# 処理遅延
process_delay = 0.1
# キューイング遅延
queue_delay = 20

# 伝搬遅延
prop_delay = (200*1000) / (2*10**8) * 1000
# 送信遅延
tx_delay = (1500 * 8) / (100*10**6) * 1000

sum([process_delay, queue_delay, prop_delay, tx_delay])

21.220000000000002

各遅延の内訳（単位は ms）

* 処理遅延: $(D_{\text{proc}} = 0.1)$
* キューイング遅延: $(D_{\text{queue}} = 20)$
* 伝搬遅延:
  ```math
  D_{\text{prop}}=\frac{\text{距離}}{\text{伝搬速度}}\\
  =\frac{200,\text{km}\times 1000,\text{m/km}}{2\times 10^8,\text{m/s}}
  =\frac{200{,}000}{200{,}000{,}000},\\
  \text{s}=0.001,\text{s}=1.0\text{ms}
  ```
* 伝送（送信）遅延:
  ```math
  D_{\text{tx}}=\frac{\text{パケット長[bit]}}{\text{レート[bit/s]}}\\
  =\frac{1500\times 8}{100\times 10^6},\text{s}
  =\frac{12{,}000}{100{,}000{,}000},\text{s}
  =0.00012,\text{s}=0.12,\text{ms}
  ```

合計（per-hop delay）
$$
D_{\text{hop}}=D_{\text{proc}}+D_{\text{queue}}+D_{\text{prop}}+D_{\text{tx}}
=0.1+20+1.0+0.12=21.22,\text{ms}
$$

# 答え

**ホップごとの遅延（per-hop delay）は 21.22 ms。**


In [20]:
def calc_hop_delay(
    proc_delay: float,
    queue_delay: float,
    link_distance: float,
    propagation_speed: float,
    pkt_size: int,
    transmission_rate: float,
) -> float:
    """
    Parameters
    ----------
    proc_delay : float
        処理遅延 (Processing delay) [s]
    queue_delay : float
        キューイング遅延 (Queueing delay) [s]
    link_distance : float
        リンクの距離 (Link distance) [m]
    propagation_speed : float
        リンクの伝搬スピード (Link propagation speed) [m/s]
    pkt_size : int
        パケットサイズ (Packet size) [bytes]
    transmission_rate : float
        リンク伝送速度 (Link transmission rate) [bps]

    Returns
    -------
    float
        1ホップあたりの遅延 (per-hop delay) [s]

    Example
    -------
    >>> per_hop_delay = calc_hop_delay(0.1/1000, 20/1000, 200*1000, 2*10**8, 1500, 100*10**6)
    >>> f"{per_hop_delay:.5f}"
    '0.02122'

    Notes
    -----
    per-hop delay D_hop は次のように計算される:
        D_hop = proc_delay + queue_delay + prop_delay + trans_delay

    - prop_delay = link_distance / propagation_speed
    - trans_delay = (pkt_size * 8) / transmission_rate
    """
    # 伝搬遅延
    prop_delay = link_distance / propagation_speed
    # 伝送遅延
    trans_delay = (pkt_size * 8) / transmission_rate
    # 合計
    return proc_delay + queue_delay + prop_delay + trans_delay
import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)

## 3. TCP輻輳制御における、スロースタートモードと輻輳回避モードについて、自分なりにまとめてください。

2つのモードは、cwnd(Congestion Window)という送信側が一度に送ってよいデータ量の上限をネットワークの混雑状況に応じて調整するためにある。
スロースタートモードのcwndの初期値は1MSSほどで、1RTTごとにcwndが2倍になる指数関数的増加をする。素早くスループットを高める目的で使用されるがネットワーク混雑の影響を受けやすい。コネクション開始時、再送信タイムアウト時に使用される。cwndがある閾値(ssthresh)に到達したら輻輳回避モードになる。
  ssthresh = if コネクション開始時 [最大ウィンドウサイズ] elif 再送信タイムアウト [その時点でのcwndの半分]
輻輳回避モードでは1RTTごとにcwndが1MSSだけ増加する線形増加をする。つまりcwnd の増え方が穏やかになる。輻輳を起こさない範囲で安定した転送を続け、負荷をかけすぎないように徐々に帯域を探るモード。